<a href="https://colab.research.google.com/github/JoDeMiro/Ember/blob/main/04_Text_Classification_with_CNN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Ajánlott ovlasmányok

<a href="http://www.joshuakim.io/understanding-how-convolutional-neural-network-cnn-perform-text-classification-with-word-embeddings/">Understanding how CNN perform Text Classification with Word Embeddings</a>


# 04 - Szöveg osztályozás Konvolúciós Hálóval.

A feladat, hogy a szövegeket (dokumentumokat) egy előre meghatározott számú halmaz valamelyik eleméhez rendeljük. Konvulúciós háló esetén az egyik problémát az jelenti, hogyha szöveg szavait vektorokká alakítjuk, akkor is különböző hosszú szövegek mátrixok álnak elő belőle.

Amit itt bemutatok az a <a href="https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf">Hierarchical Attention Networks for Document Classification</a> cikk alapján előállított model. De sorra veszem a két másik eljárást a CNN-t és az LSTM-t is.


In [2]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Concatenate, Dropout
from keras.models import Model

# Erre most nincs szükségem de itt hagyom

import nltk
nltk.download('punkt')

In [3]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

Megtísztítom a szövegtörzset a sortörésektől vagy a html tagoktól, továbbá elvégzem a szükséges átalakításokat például minden char legyen lowercase, etc.

In [4]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()


In [9]:
# download imdb train and keep the files in the working directory
!wget https://github.com/prateek22sri/Sentiment-analysis/raw/master/labeledTrainData.tsv

--2022-01-25 08:50:31--  https://github.com/prateek22sri/Sentiment-analysis/raw/master/labeledTrainData.tsv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/prateek22sri/Sentiment-analysis/master/labeledTrainData.tsv [following]
--2022-01-25 08:50:31--  https://raw.githubusercontent.com/prateek22sri/Sentiment-analysis/master/labeledTrainData.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33556378 (32M) [text/plain]
Saving to: ‘labeledTrainData.tsv’

labeledTrainData.ts 100%[===================>]  32.00M  --.-KB/s    in 0.1s    

2022-01-25 08:50:31 (302 MB/s) - ‘labeledTrainData.tsv’ saved [

In [6]:
# download glove word vector
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2022-01-25 08:44:27--  https://www.kaggle.com/c/word2vec-nlp-tutorial/download/labeledTrainData.tsv
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?returnUrl=%2Fc%2Fword2vec-nlp-tutorial [following]
--2022-01-25 08:44:27--  https://www.kaggle.com/account/login?returnUrl=%2Fc%2Fword2vec-nlp-tutorial
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘labeledTrainData.tsv’

labeledTrainData.ts     [ <=>                ]   6.43K  --.-KB/s    in 0s      

2022-01-25 08:44:27 (82.1 MB/s) - ‘labeledTrainData.tsv’ saved [6585]

--2022-01-25 08:44:27--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP

In [81]:
data_train = pd.read_csv('labeledTrainData.tsv', sep='\t')
print(data_train.shape)

texts = []
labels = []

for idx in range(data_train.review.shape[0]):
    text = BeautifulSoup(data_train.review[idx], "lxml")
    texts.append(clean_str(text.get_text()))
    labels.append(data_train.sentiment[idx])
    

(25000, 3)


In [82]:
print('len(texts) = ', len(texts))
print('len(labels) = ', len(labels))

len(texts) =  25000
len(labels) =  25000


In [83]:
print('MAX_NB_WORDS = ', MAX_NB_WORDS)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

MAX_NB_WORDS =  20000
Found 81501 unique tokens.


In [84]:
print('type(sequences) = ', type(sequences))
print('len(sequences) = ', len(sequences))
print('type(sequences[0]) = ', type(sequences[0]))
print('len(sequences[0]) = ', len(sequences[0]))    # első dok/szöveg hossza
print('len(sequences[1]) = ', len(sequences[1]))    # második dok/szöveg hossza
print('len(sequences[2]) = ', len(sequences[2]))    # harmadik dok/szöveg hossza
print('sequences[0][:5] = ', sequences[0][:5])      # első dok első 5 szava
print('sequences[0][0] = ', sequences[0][0])           # első dok, első szó
print('sequences[0][1] = ', sequences[0][1])           # első dok, második szó
print('sequences[0][2] = ', sequences[0][2])           # első dok, harmadik szó
print('type(word_index) = ', type(word_index))

# First doc, first word
print(list(word_index.keys())[list(word_index.values()).index(sequences[0][0])])
print(list(word_index.keys())[list(word_index.values()).index(sequences[0][1])])
print(list(word_index.keys())[list(word_index.values()).index(sequences[0][2])])


type(sequences) =  <class 'list'>
len(sequences) =  25000
type(sequences[0]) =  <class 'list'>
len(sequences[0]) =  419
len(sequences[1]) =  160
len(sequences[2]) =  352
sequences[0][:5] =  [15, 29, 10, 537, 165]
sequences[0][0] =  15
sequences[0][1] =  29
sequences[0][2] =  10
type(word_index) =  <class 'dict'>
with
all
this


Korábban utaltam rá, hogy ha számokká alakított szövegeken olyan osztályozást szeretnénk végrehajtani amelynek az alapját a konvolúciós háló végzi, akkor szükséges az egyes osztályozásra/kategorizálásra váró szövegeket azon méretűre hozni.

Hogy néz ki az azonos méretre hozás ilyen szövegek esetében. Itt a *sorok* száma a szavak száma az adott dokumentumban. Az *oszlopok* száma fix, ez a vektorrá alakított szavak hosszának felel meg, amit a programunk elején megadtunk. Ebben a példaprogram ezt az értéket a `EMBEDDING_DIM = 100` értékben határoztam meg.

Fontos kitétel, hogy a szavak vektorrá alakítását nem én és most végeztem el, hanem egy széles körben használt angol nyelvű **Word2Vec** modelt használtam fel. Ugyanakkor fontos megérteni, hogy például egy másik kutatásban az így előállított alacsonyabb dimenziójú reprezentációt (word_embeding) és én állítottam el. Ennek a tanítása ugyanakkor időigényes, és nagy adathalmazt igényel. Mivel ennek a bemutatónak a célja a Konvolúció bemutatása a szövegosztályozásban ezért a **Word2Vec** transformációtól itt most eltenkintek.

## Szükséges átalakítás a konvolúcióhoz

Tehát, hogy osztályozni tudja a dokumentumokat azonos méretű mátrixokká kell alakítanom őket. Mive az oszlopok száma adott, csak a sorok száma változik, a sorok számát kell azonos értékre hoznom.

Erre többféle módszer is van, most ebbe nem megyek bele.

In [76]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [85]:
one_hot_labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', one_hot_labels.shape)

Shape of data tensor: (25000, 1000)
Shape of label tensor: (25000, 2)


In [87]:
labels[0]

1

In [86]:
one_hot_labels[0]

array([0., 1.], dtype=float32)

## Labet to One-Hot-Vector

Eredetileg két kategóriánk volt, ugyhogy az ebből létrehozott One-Hot-Vector is kettő hosszú lesz. Több kategória esetén értelemszerűen hosszabb lenne.

In [90]:
labels.count(0)

12500

In [100]:

def zero_rule_algorithm_classification(sample):
  uniqueValues = np.unique(sample)
  counts = []
  for i in uniqueValues:
    count = sample.count(i) / len(sample)
    counts.append(count)
  
  return counts

In [101]:
zero_rule_algorithm_classification(labels)

[0.5, 0.5]

## Zero Rule Classifier

By the way, mindíg jó öltelt megnézni, hogy mi a kimeneit változó eloszlása. Ez egy referencia pont, hogy mihez képest kell majd a modellemnek jobban teljesítenie.

In [104]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
one_hot_labels = one_hot_labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

## Shuffle the data

Keverjük össze az adatokat, hogy véletlenszerűen válaszunk belőlük a tanuló és a teszt mintába is.

In [17]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set ')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in traing and validation set 
[ 9973. 10027.]
[2527. 2473.]


In [20]:
GLOVE_DIR = "glove/glove.6B/"
GLOVE_DIR = ""
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [21]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [22]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [23]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - simplified convolutional neural network")
model.summary()

model fitting - simplified convolutional neural network
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1000)]            0         
                                                                 
 embedding (Embedding)       (None, 1000, 100)         8150200   
                                                                 
 conv1d (Conv1D)             (None, 996, 128)          64128     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 199, 128)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 195, 128)          82048     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 39, 128)          0         
 1D) 

In [26]:
# model.fit(x_train, y_train, validation_data=(x_val, y_val),
#           epochs=10, batch_size=128)

# Ez nagyon sokáig tartana, úgyhogy most egy kicsit visszaveszek
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=1, batch_size=128)


157/157 [==============================] - 246s 2s/step - loss: 0.6116 - acc: 0.6615 - val_loss: 0.4378 - val_acc: 0.8040


In [27]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [28]:
# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

In [29]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

In [ ]:
Conv1D()

In [31]:
for fsz in filter_sizes:
    l_conv = Conv1D(128, fsz, activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)

In [32]:
l_merge = Concatenate(axis=1)(convs)
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - more complex convolutional neural network")
model.summary()

model fitting - more complex convolutional neural network
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1000)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 1000, 100)    8150200     ['input_2[0][0]']                
                                                                                                  
 conv1d_3 (Conv1D)              (None, 998, 128)     38528       ['embedding_1[0][0]']            
                                                                                                  
 conv1d_4 (Conv1D)              (None, 997, 128)     51328       ['embedding_1[0][0]']            
                                  

In [35]:
# model.fit(x_train, y_train, validation_data=(x_val, y_val),
#           nb_epoch=20, batch_size=50)

# Ez kicsit sokáig tartana, úgyhogy most egy kicsit visszaveszek
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=1, batch_size=50)

400/400 [==============================] - 583s 1s/step - loss: 0.5372 - acc: 0.7102 - val_loss: 0.3295 - val_acc: 0.8564
